In [1]:
# importeer de benodigde bibliotheken
import os
import pandas as pd
from pathlib import Path

# Vanuit toolbox gebruiken we standaard 'Config' en 'DataAdapter'.
# De functie die we willen gebruiken om data op te halen is 'LoadsWaterinfo'
from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter
from toolbox_continu_inzicht.loads import LoadsWaterinfo, LoadsClassify, get_waterinfo_locations, get_waterinfo_thresholds

# Hiermee kan je markdown printen tijdens uitvoeren van functie
from IPython.display import display, Markdown

In [2]:
# ophalen waterstanden
config_file = Path("data_sets/loads_waterinfo_config_workflow_1.yaml")
if os.path.exists(config_file):
    config = Config(config_path=config_file)
    config.lees_config()

    data_adapter = DataAdapter(config=config)
else:
    display(Markdown(f"**⚠️ Config file '{config_file}' not found.**"))


In [ ]:
# bepaal input, output en datatype voor volgende stap
adapter = config.data_adapters

options = config.global_variables["LoadsWaterinfo"]
datatype = options["datatype"]

# invoerbestand locaties
file_locations = Path(
    config.global_variables["rootdir"] / Path(adapter["locaties"]["path"])
)
if os.path.exists(file_locations):
    os.remove(file_locations)

# uitvoerbestand waterstanden
file_water_levels = Path(
    config.global_variables["rootdir"] / Path(adapter["waterstanden"]["path"])
)
if os.path.exists(file_water_levels):
    os.remove(file_water_levels)    

# uitvoerbestand waterstanden
file_thresholds = Path(
    config.global_variables["rootdir"] / Path(adapter["thresholds"]["path"])
)
if os.path.exists(file_thresholds):
    os.remove(file_thresholds)    

display(Markdown(f"** datatype: '{datatype}' **"))
display(Markdown(f"** locatiebestand: '{file_locations}' **"))
display(Markdown(f"** waterstandenbestand: '{file_water_levels}' **"))
display(Markdown(f"** thresholdsbestand: '{file_thresholds}' **"))

In [ ]:
# ophalen locaties
df_locations = await get_waterinfo_locations(parameter_id=datatype)
df_locations

In [5]:
locations=["Aadorp","Houtrib","Vliss","Hoek","Lobith"]
df_selection = df_locations[df_locations["name"].str.contains("|".join(locations))]
df_selection = df_selection[["name","location_code"]].rename(columns={"location_code":"code"})

# schrijf locaties naar csv bestand
df_selection.to_csv(file_locations)

In [6]:
# waterstanden opslaan
waterinfo = LoadsWaterinfo(data_adapter=data_adapter, input="locaties", output="waterstanden")
df_out = await waterinfo.run()

In [7]:
# thresholds ophalen

df_all_thresholds = pd.DataFrame()

for _, location in df_selection.iterrows():
  df_thresholds = await get_waterinfo_thresholds(location["code"], parameter_id=datatype)
  df_thresholds = df_thresholds[["from","to","color","label"]]
  df_thresholds = df_thresholds.rename(columns={"from":"van","to":"tot","color":"kleur"})
  df_thresholds["code"] = location["code"]
  
  df_all_thresholds = pd.concat([df_all_thresholds,df_thresholds])

# thresholds opslaan
df_all_thresholds.to_csv(file_thresholds)

In [8]:
classify = LoadsClassify(data_adapter=data_adapter,input="thresholds",input2="waterstanden",output="classificatie")
df_classify = await classify.run()
